In [ ]:
import os
from glob import glob
import numpy as np
from tqdm import tqdm

from models.VAE import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
import os
import glob

###### http://marsyas.info/downloads/datasets.html
###### GTZAN Genre Collection

In [ ]:
MUSIC_FOLDER = './VAE_MusicComposer/data'

In [ ]:
DATA_FOLDER1 = './VAE_MusicComposer/data/blues'
DATA_FOLDER2 = './VAE_MusicComposer/data/classical'
DATA_FOLDER3 = './VAE_MusicComposer/data/country'
DATA_FOLDER4 = './VAE_MusicComposer/data/disco'
DATA_FOLDER5 = './VAE_MusicComposer/data/hiphop'
DATA_FOLDER6 = './VAE_MusicComposer/data/jazz'
DATA_FOLDER7 = './VAE_MusicComposer/data/metal'
DATA_FOLDER8 = './VAE_MusicComposer/data/pop'
DATA_FOLDER9 = './VAE_MusicComposer/data/reggae'
DATA_FOLDER10 = './VAE_MusicComposer/data/rock'
DATA = [DATA_FOLDER1,
        DATA_FOLDER2,
        DATA_FOLDER3,
        DATA_FOLDER4,
        DATA_FOLDER5,
        DATA_FOLDER6,
        DATA_FOLDER7,
        DATA_FOLDER8,
        DATA_FOLDER9,
        DATA_FOLDER10]
CLASSES = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

In [ ]:
#spectrogram dimensions
# 160, 1024
INPUT_DIM = (160,1024,1)

In [ ]:
INPUT_DIM

In [ ]:
INPUT_DIM[:2]

In [ ]:
# run params
section = 'music'
run_id = '0001'
data_name = 'musicdata'
RUN_FOLDER = 'VAE_MusicComposer/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

In [ ]:
nb_train_samples = 1000
batch_size = 1
epochs = 100
path_to_data = './VAE_MusicComposer/data/'

In [ ]:
lendata = len(DATA)
lendata

In [ ]:
source_images = []

for i in range(lendata):
    path = DATA[i]
    class_label = CLASSES[i]
    files = os.listdir(path)
    for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
        source_images.append(os.path.join(filename))

In [ ]:
source_images[0]

In [ ]:
from random import shuffle
shuffle(source_images)
len(source_images)

In [ ]:
def data_gen():
    
    while True:
        for start in range(0, nb_train_samples, batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, nb_train_samples)
            for img_path in range(start, end):
#                 print(source_images[img_path])
                img = np.load(source_images[img_path])
#                 print(img.shape)
                img = np.reshape(img,(img.shape[0],img.shape[1],1))
                x_batch.append(img)
                if CLASSES[0] in source_images[img_path]:
                    y_batch.append(["0"])
                elif CLASSES[1] in source_images[img_path]:
                    y_batch.append(["1"])
                elif CLASSES[2] in source_images[img_path]:
                    y_batch.append(["2"])
                elif CLASSES[3] in source_images[img_path]:
                    y_batch.append(["3"])
                elif CLASSES[4] in source_images[img_path]:
                    y_batch.append(["4"])
                elif CLASSES[5] in source_images[img_path]:
                    y_batch.append(["5"])
                elif CLASSES[6] in source_images[img_path]:
                    y_batch.append(["6"])
                elif CLASSES[7] in source_images[img_path]:
                    y_batch.append(["7"])
                elif CLASSES[8] in source_images[img_path]:
                    y_batch.append(["8"])
                elif CLASSES[9] in source_images[img_path]:
                    y_batch.append(["9"])
            
            yield (np.array(x_batch), np.array(y_batch))

In [ ]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,1]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=50
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [ ]:
vae.encoder.summary()

In [ ]:
vae.decoder.summary()

In [ ]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 500
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [ ]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [ ]:
vae.train_with_generator(     
    data_gen(), 
    epochs= epochs,
    steps_per_epoch= nb_train_samples // batch_size
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)